In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
import uuid
import torch

class BaseModel:
    def __init__(self, uuid, model_name="Qwen/Qwen2.5-3B-Instruct", tokenizer_name="Qwen/Qwen2.5-3B-Instruct",
                 max_tokens = 512, history_limit=5):
        self.DIALOGS_DIR = "dialogues"
        os.makedirs(self.DIALOGS_DIR, exist_ok=True)

        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.max_tokens = max_tokens 
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            device_map="auto"
        ).to(self.device)
        self.model = self.model.half().to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.uuid = uuid
        self.history_limit = history_limit
        self.history = self.load_history()
        print(self.model.device)

        
    def load_history(self):
        history_path = os.path.join(self.DIALOGS_DIR, f"{self.uuid}.json")
        if os.path.exists(history_path):
            with open(history_path, "r", encoding="utf-8") as f:
                return json.load(f)
        return [{"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."}]

    def save_history(self):
        history_path = os.path.join(self.DIALOGS_DIR, f"{self.uuid}.json")
        with open(history_path, "w", encoding="utf-8") as f:
            json.dump(self.history, f, ensure_ascii=False, indent=4)


    def ask(self, question):
        """Добавляет вопрос пользователя, обновляет историю и отправляет запрос модели."""
        self.history.append({"role": "user", "content": question})
        self.save_history()

        text = self.tokenizer.apply_chat_template(
            self.history,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=self.max_tokens
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        self.history.append({"role": "assistant", "content": response})
        self.save_history()

        return response

In [2]:
dialog_id = str(uuid.uuid4())
model = BaseModel(dialog_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

cuda:0


In [4]:
print(model.history)
print("Ответ модели:", model.ask("А как тебя зовут?"))

[{'role': 'system', 'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.'}, {'role': 'user', 'content': 'Что ты можешь?'}, {'role': 'assistant', 'content': 'Я могу общаться на различные темы, отвечать на вопросы, помогать с информацией, выполнять задачи и задавать вопросы по вашему запросу. Также я могу генерировать текст на разные темы, включая литературный и научный. У меня есть база знаний, которая обновляется ежедневно, поэтому я могу давать актуальные ответы и информацию.'}]


KeyboardInterrupt: 

In [3]:
print(model.history)
print("Ответ модели:", model.ask("Что ты можешь?"))

[{'role': 'system', 'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.'}]
Ответ модели: Я могу общаться на различные темы, отвечать на вопросы, помогать с информацией, выполнять задачи и задавать вопросы по вашему запросу. Также я могу генерировать текст на разные темы, включая литературный и научный. У меня есть база знаний, которая обновляется ежедневно, поэтому я могу давать актуальные ответы и информацию.


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Что ты можешь? "
messages = [{'role': 'system', 'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.'}, 
            {'role': 'user', 'content': 'Что ты можешь?'}, 
            {'role': 'assistant', 'content': 'Я могу общаться на различные темы, отвечать на вопросы, помогать с информацией, выполнять задачи и задавать вопросы по вашему запросу. Также я могу генерировать текст на разные темы, включая литературный и научный. У меня есть база знаний, которая обновляется ежедневно, поэтому я могу давать актуальные ответы и информацию.'},
            {'role': 'user', 'content': 'А как тебя зовут?'},
            {'role': 'assistant', 'content': "Меня создали для общения, но у меня нет имени в обычном понимании этого слова. Вы можете обращаться ко мне просто как Qwen или использовать любое другое имя, которое вам будет приятно."},
            {'role': 'user', 'content': 'Можно я буду назвыать тебя EZA?'},
            {'role': 'assistant', 'content': 'Конечно, вы можете называть меня как угодно. Если вам удобно обращаться ко мне как EZA, то это прекрасно!'},
            {'role': 'user', 'content': "Какое было моё первое сообщение?"},
            ]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

К сожалению, как искусственный интеллект, у меня нет доступа к историческим диалогам наших предыдущих сообщений. Я начинаю каждую новую беседу с чистого состояния, не имея информации о предыдущих взаимодействиях. Но вы можете начать общение с любого вопроса или темы, и мы сможем продолжить оттуда.
